In [ ]:
import treams as tr
import numpy as onp
import matplotlib.pyplot as plt

In [ ]:
basis = tr.SphericalWaveBasis.default(3)

In [ ]:
def to_spherical_coordinates(x,y,z):
    r = onp.sqrt(x**2+y**2+z**2)
    theta = onp.arccos(z/r)
    phi = onp.sign(y)*onp.arccos(x/onp.sqrt(x**2+y**2))
    theta[r==0] = 0
    phi[r==0] = 0
    return r, theta, phi


In [ ]:
x = y = z = onp.linspace(-4*onp.pi, 4*onp.pi)
positions = onp.meshgrid(x, y, z)

In [ ]:
basis[0]

In [ ]:
basis[1]

In [ ]:
for input_vector in basis:
    positions_spherical = to_spherical_coordinates(*positions)
    field = tr.special.vsw_rA(input_vector[1], input_vector[2], *positions_spherical, input_vector[3])
    fig, axs = plt.subplots(1, 3, figsize=(6,2))
    for i, ax in enumerate(axs):
        ax.contourf(onp.real(field[:,:,25,i]),vmin = -0.1, vmax = 0.1, cmap="RdBu")
        ax.axis("off")
    plt.suptitle(input_vector)


$$\nabla\times\left(\frac{1}{\mu(\mathbf{r})}\nabla\times\mathbf{E}_{\mathrm{scat}}\left(\mathbf{r}\right)\right)-\omega^{2}\epsilon\left(\mathbf{r}\right)\mathbf{E}_{\mathrm{scat}}\left(\mathbf{r}\right)=\nabla\times\left(\frac{1}{\mu(\mathbf{r})}\nabla\times\mathbf{E}_{\mathrm{inc}}\left(\mathbf{r}\right)\right)-\omega^{2}\epsilon\left(\mathbf{r}\right)\mathbf{E}_{\mathrm{inc}}\left(\mathbf{r}\right).$$

$$\nabla\times\left(\frac{1}{\mu(\mathbf{r})}\nabla\times\mathbf{E}_{\mathrm{scat}}\left(\mathbf{r}\right)\right)-\omega^{2}\epsilon\left(\mathbf{r}\right)\mathbf{E}_{\mathrm{scat}}\left(\mathbf{r}\right)=\omega^{2}\mathbf{E}_{\mathrm{inc}}\left(\mathbf{r}\right)\left[\epsilon_{bg}-\epsilon_{fg}\left(\mathbf{r}\right)\right]$$

In [ ]:
import treams

In [ ]:
k0 = 2 * onp.pi * 1/1550
materials = [treams.Material(12), treams.Material(2)]
lmax = 1
radius = 20*40
spheres = [treams.TMatrix.sphere(lmax, k0, radius, materials)]

In [ ]:
basis = tr.SphericalWaveBasis.default(3)
mode = basis[5]

In [ ]:
tm = spheres[-1]
inc = treams.spherical_wave(mode[1], mode[2], mode[3], k0=tm.k0, material=tm.material, poltype="helicity")
sca = tm @ inc.expand(tm.basis)
grid = onp.mgrid[-2000:2000:401j, -2000:2000:401j, 0:1].squeeze().transpose((1, 2, 0))
valid = tm.valid_points(grid, [radius])
field = sca.efield(grid[valid])

In [ ]:
plt.figure(figsize=(6,2))
for i in range(3):
    plt.subplot(131+i)
    intensity = onp.zeros_like(grid[..., 0])
    intensity[~valid] = onp.nan
    intensity[valid] = onp.abs(field[:,i])
    pcm = plt.pcolormesh(
        grid[:, 0, 0], grid[0, :, 1], intensity, shading="nearest", vmin=0, vmax=0.03
    )
    plt.axis("off")
    